In [1]:
import requests
from requests import Response
from bs4 import BeautifulSoup, Tag
import pandas as pd

In [6]:


# Scrapes the requirement elements from the page content provided
def scrape_page(response: Response) -> None:

    # Parse site's content html
    content: bytes = response.content
    soup: Beautifulsoup =  BeautifulSoup(content, 'html.parser')
    
    # Get the books from the list
    ol = soup.find('ol')
    articles = ol.find_all('article', class_='product_pod')
    
    data = []
    # Extract the fields needed
    for article in articles:
        image = article.find('img')
        title = image.attrs['alt']
        star_elem = article.find('p')
        star_num = star_elem.attrs['class'][1]
        price = article.find('p', class_='price_color').get_text()
        price_float = float(price[1:])
    
        data.append([title, price_float, star_num])
        
    return data
        
        
    
# Gets page and hanlde errors
def get_page(url: str) -> Response:
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as e:
        print(f"Error with requests for {url}: {e}", "Aborting...!")
        raise SystemExit(e)

In [7]:
from typing import Any, List
from requests.models import Response

books: List[Any] = []
# Get all pages and scrape them one by one while handling any errors
for i in range(1, 51):
    URL: str = f"http://books.toscrape.com/catalogue/page-{i}.html"
    response = get_page(URL)

    if response:
        data = scrape_page(response)
        books.extend(data)

In [4]:
df = pd.DataFrame(books, columns=['Title','Price', 'Star Rating'])
df.to_csv('books.csv')
print("Done!")

Done!
